<a href="https://colab.research.google.com/github/Sina-Akhavi/bitcoin-timeseries-Forecasting/blob/main/GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import math
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
import os
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers



# os.chdir('./sample_data')

df = pd.read_csv('BTC-USD.csv')

X_df = df.drop(['Adj Close'], axis='columns')

X_df = X_df[X_df['Date'] > '2014-09-17']
X_df = X_df[X_df['Date'] < '2020-08-01']
X_df = X_df.drop(['Open', 'High', 'Low', 'Volume'], axis='columns')
close_stock = X_df.copy()

del X_df['Date']
scaler = MinMaxScaler()
closedf = scaler.fit_transform(np.array(X_df).reshape(-1, 1))

training_size = int(len(closedf) * 0.80)
test_size = len(closedf) - training_size
train_data, test_data = closedf[0: training_size, :], closedf[training_size: len(closedf), :1]

# train_data[:5]
print('train_data shape: ', train_data.shape)
print('test_data shape: ', test_data.shape)

def create_XtrainYtrain(dataset, time_step):

  data_X, data_Y = [], []

  for i in range(len(dataset) - time_step - 1):
    a = dataset[i: (i + time_step), 0]
    data_X.append(a)
    data_Y.append(dataset[(i + time_step), 0])

  return np.array(data_X), np.array(data_Y)



time_step = 15
X_train, y_train = create_XtrainYtrain(train_data, time_step)
X_test, y_test = create_XtrainYtrain(test_data, time_step)


print('x_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('x_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)


X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

print('X_train.shape: ', X_train.shape)
print('X_test.shape: ', X_test.shape)


model = models.Sequential([
layers.GRU(31, input_shape=(time_step, 1), activation='relu',return_sequences=True),
layers.GRU(31, activation='relu'),
layers.Dropout(0.5),
layers.Dense(1)
])


model.summary()

model.compile(loss="mean_squared_error", optimizer="adam")




model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_test, y_test),verbose=1)

train_data shape:  (1715, 1)
test_data shape:  (429, 1)
x_train shape: (1699, 15)
y_train shape: (1699,)
x_test shape: (413, 15)
y_test shape: (413,)
X_train.shape:  (1699, 15, 1)
X_test.shape:  (413, 15, 1)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 15, 31)            3162      
                                                                 
 gru_1 (GRU)                 (None, 31)                5952      
                                                                 
 dropout (Dropout)           (None, 31)                0         
                                                                 
 dense (Dense)               (None, 1)                 32        
                                                                 
Total params: 9146 (35.73 KB)
Trainable params: 9146 (35.73 KB)
Non-trainable params: 0 (0.00 Byte)
____________

In [8]:
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# do the inverse transformation

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)

original_ytrain = scaler.inverse_transform(y_train.reshape(-1, 1))
original_ytest = scaler.inverse_transform(y_test.reshape(-1, 1))

13/13 [==============================] - 0s 6ms/step


In [9]:
def calculate_mape(actual, predicted) -> float:

    # Convert actual and predicted
    # to numpy array data type if not already
    if not all([isinstance(actual, np.ndarray),
                isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)

    # Calculate the MAPE value and return
    return round(np.mean(np.abs((
      actual - predicted) / actual)) * 100, 2)



print('Train Data RMSE: ', math.sqrt(mean_squared_error(original_ytrain, train_predict)))
print('Train Data MSE: ', mean_squared_error(original_ytrain, train_predict))
print('Train Data MAE: ', mean_absolute_error(original_ytrain, train_predict))
print('Train MAPE=', calculate_mape(original_ytrain, train_predict))
print('----------------------------------------------------------')
print('Test Data RMSE: ', math.sqrt(mean_squared_error(original_ytest, test_predict)))
print('Test Data MSE: ', mean_squared_error(original_ytest, test_predict))
print('Test Data MAE: ', mean_absolute_error(original_ytest, test_predict))
print('Test MAPE=', calculate_mape(original_ytest, test_predict))

Train Data RMSE:  361.9934912779993
Train Data MSE:  131039.28772763496
Train Data MAE:  261.8696650552393
Train MAPE= 41.96
----------------------------------------------------------
Test Data RMSE:  469.4454732585478
Test Data MSE:  220379.0523629419
Test Data MAE:  363.9476368460957
Test MAPE= 4.04


In [12]:
# shift train predictions for plotting
from itertools import cycle
import plotly.express as px

look_back = time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back: len(train_predict) + look_back, :] = train_predict
print("Train Predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict) + (look_back * 2) + 1: len(closedf)-1, :] = test_predict
print("Test Predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price', 'Train Predicted close price',  'Test Predicted close price'])

plotdf = pd.DataFrame({'date': close_stock['Date'],
                       'original_close': close_stock['Close'],
                       'train_predicted_close': trainPredictPlot.reshape(1, -1)[0].tolist(),
                       'test_predicted_close': testPredictPlot.reshape(1, -1)[0].tolist()})

fig = px.line(plotdf, x=plotdf['date'], y=[plotdf['original_close'], plotdf['train_predicted_close'], plotdf['test_predicted_close']],
              labels={'value': 'stock price', 'date': 'Data'})

fig.update_layout(title_text='Comparison between original close price vs predicted close price', plot_bgcolor='white', font_size=15,
                  font_color='black', legend_title_text='Close price')

fig.for_each_trace(lambda t: t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()


Train Predicted data:  (2144, 1)
Test Predicted data:  (2144, 1)
